In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./env/.env")

api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Chat 모델 및 프롬프트 설정
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# ChatOpenAI 모델 초기화
chat = ChatOpenAI(
    temperature= 0.1, # 낮을수록 일관된 답변, 높을수록 창의적인 답변
    streaming=True, # 답변이 생성되는 과정을 실시간으로 확인
    callbacks=[StreamingStdOutCallbackHandler()] 
)   

# Few-Shot Learning을 위한 예제 데이터
# AI에게 원하는 답변 형식을 학습시키기 위한 예시들
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 1. 하나의 예제 형식을 정의
# FewShotChatMessagePromptTemplate에게 전달됨
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?" ),
        ("ai", "{answer}")
    ]
)


# 2. 여러 예제들을 이 형식으로 변환
# examples 리스트의 각 예제를 1번 형식에 맞춰서 변환
# 결과: 3개의 예제가 모두 메시지 형식으로 변환 됨
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt, # 1번에서 만든 틀 사용
    examples = examples, # 실제 예제 데이터
)
# -> 3개의 예제가 모두 ("human", ...) ("ai", ...) 형태로 변환됨


# 3. 전체 대화 구조 조립
# 시스템 메시지 + 변환된 예제들 + 실제 질문들 합쳐서
# AI에게 보낼 최종 프롬프트 완성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."), # 시스템 역할 정의
    example_prompt, # 2번에서 만든 예제 3개 전부
    ("human", "What do you know about {country}?") # 실제 사용자 질문
])
# -> AI에게 전달될 최종 형태

# LangChain Expression Language를 사용할 체인 구성
chain = final_prompt | chat

# 체인 실행: Japan에 대한 정보 요청
chain.invoke({
    "country": "Japan"
})


        I know this:
        Capital: Tokyo
        Language: Japanese
        Food: Sushi and Ramen
        Currency: Yen

AIMessageChunk(content='\n        I know this:\n        Capital: Tokyo\n        Language: Japanese\n        Food: Sushi and Ramen\n        Currency: Yen')